In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from warnings import filterwarnings
from mpl_toolkits.mplot3d import Axes3D
import statsmodels.api as sm
import missingno as msno
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.neighbors import LocalOutlierFactor
from scipy.stats import levene
from scipy.stats import shapiro
from scipy.stats.stats import pearsonr
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.preprocessing import scale
from sklearn.model_selection import ShuffleSplit, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import model_selection
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LinearRegression
from sklearn.cross_decomposition import PLSRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import ElasticNetCV
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
import xgboost as xgb
from xgboost import XGBRegressor, XGBClassifier
from lightgbm import LGBMRegressor, LGBMClassifier
from catboost import CatBoostRegressor, CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn import tree
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_auc_score, roc_curve

In [ ]:
filterwarnings("ignore")

# DATA SOURCE

## All Data

In [ ]:
IC40 = pd.read_csv("../input/muon-neutrino-effective-areas/IC40_effectiveArea - IC40_effectiveArea.csv")
print(IC40.shape)
print("---"*30)
IC59 = pd.read_csv("../input/muon-neutrino-effective-areas/IC59_effectiveArea - IC59_effectiveArea.csv")
print(IC59.shape)
print("---"*30)
IC79 = pd.read_csv("../input/muon-neutrino-effective-areas/IC79_effectiveArea - IC79_effectiveArea.csv")
print(IC79.shape)
print("---"*30)
IC86II = pd.read_csv("../input/muon-neutrino-effective-areas/IC86_II_effectiveArea - IC86_II_effectiveArea.csv")
print(IC86II.shape)
print("---"*30)
IC86I = pd.read_csv("../input/muon-neutrino-effective-areas/IC86_I_effectiveArea - IC86_I_effectiveArea.csv")
print(IC86I.shape)
print("---"*30)

## Combined Data

In [ ]:
Neutrino = pd.concat([IC40,IC59,IC79,IC86II,IC86I],axis=0)
print(Neutrino.shape)
data = Neutrino.copy()

# DATA INFORMATION

In [ ]:
print(data.shape)
print("---"*30)
print(data.columns)
print("---"*30)
print(data.info())
print("---"*30)
print(data.describe().T)
print("---"*30)
print(data.corr())
print("---"*30)
print(data.isnull().sum())
print("---"*30)
print(data.where(data["E_nu/GeV_min"] < 2.8).value_counts().index)
print("---"*30)
print(data.where(data["A_Eff_cm2"] < 100000).value_counts().index)
print("---"*30)
print(data.where(data["Dec_nu_min_deg"] < -50).value_counts().index)
print("---"*30)

## IGNORING 0 VALUES

In [ ]:
data = data[data.A_Eff_cm2 != 0]

# CORRELATION - NORMALITY - HOMOGENEITY

## CORRELATION

In [ ]:
corrPearson = data.corr(method="pearson")
corrSpearman = data.corr(method="spearman")

In [ ]:
figure = plt.figure(figsize=(15,8))
sns.heatmap(corrPearson,annot=True,vmin=-1,vmax=1,center=0,cmap="coolwarm")
plt.title("PEARSON CORR")
plt.xlabel("COLUMNS")
plt.ylabel("COLUMNS")
plt.show()

In [ ]:
figure = plt.figure(figsize=(15,8))
sns.heatmap(corrSpearman,annot=True,vmin=-1,vmax=1,center=0,cmap="coolwarm")
plt.title("PEARSON CORR")
plt.xlabel("COLUMNS")
plt.ylabel("COLUMNS")
plt.show()

## NORMALITY

In [ ]:
for i in data.columns:
    print(i,"----"*30)
    print("%.4f - %.4f" % shapiro(data[i]))

## HOMOGENEITY

In [ ]:
print(levene(data["E_nu/GeV_min"],data["E_nu/GeV_max"],data["Dec_nu_min_deg"],
            data["Dec_nu_max_deg"],data["A_Eff_cm2"]))

# VISUALIZATION

In [ ]:
data.hist(figsize=(20,10))
plt.show()

In [ ]:
figure = plt.figure(figsize=(20,8))
sns.barplot(x="E_nu/GeV_min",y="A_Eff_cm2",data=data,color="black")
plt.show()

In [ ]:
figure = plt.figure(figsize=(20,8))
sns.barplot(x="E_nu/GeV_max",y="A_Eff_cm2",data=data,color="black")
plt.show()

In [ ]:
figure = plt.figure(figsize=(20,8))
sns.barplot(x="Dec_nu_min_deg",y="A_Eff_cm2",data=data,color="black")
plt.show()

In [ ]:
figure = plt.figure(figsize=(20,8))
sns.barplot(x="Dec_nu_max_deg",y="A_Eff_cm2",data=data,color="black")
plt.show()

In [ ]:
figure = plt.figure(figsize=(20,8))
sns.histplot(x="E_nu/GeV_min",data=data)
plt.show()

In [ ]:
figure = plt.figure(figsize=(20,8))
sns.histplot(x="E_nu/GeV_max",data=data)
plt.show()

In [ ]:
figure = plt.figure(figsize=(20,8))
sns.histplot(x="E_nu/GeV_min",data=data)
plt.show()

In [ ]:
figure = plt.figure(figsize=(20,8))
sns.histplot(x="Dec_nu_max_deg",data=data)
plt.show()

In [ ]:
figure = plt.figure(figsize=(20,8))
sns.histplot(x="Dec_nu_min_deg",data=data)
plt.show()

In [ ]:
figure = plt.figure(figsize=(20,8))
sns.histplot(x="A_Eff_cm2",data=data)
plt.xlim(0,10000000)
plt.show()

In [ ]:
figure = plt.figure(figsize=(20,10))
sns.displot(
    data=data,
    x="Dec_nu_min_deg",
    kind="kde", height=6, clip=(0, None),
)
plt.show()

In [ ]:
figure = plt.figure(figsize=(20,10))
sns.displot(
    data=data,
    x="Dec_nu_max_deg",
    kind="kde", height=6, clip=(0, None),
)
plt.show()

In [ ]:
figure = plt.figure(figsize=(20,10))
sns.displot(
    data=data,
    x="E_nu/GeV_min",
    kind="kde", height=6, clip=(0, None),
)
plt.show()

In [ ]:
figure = plt.figure(figsize=(20,10))
sns.displot(
    data=data,
    x="E_nu/GeV_max",
    kind="kde", height=6, clip=(0, None),
)
plt.show()

In [ ]:
figure = plt.figure(figsize=(20,10))
sns.displot(
    data=data,
    x="A_Eff_cm2",
    kind="kde", height=6, clip=(0, None),
)
plt.show()

In [ ]:
figure = plt.figure(figsize=(20,10))
sns.relplot(x="E_nu/GeV_min", y="A_Eff_cm2", sizes=(40, 400), alpha=.2, color="r",height=10, data=data)
plt.show()

In [ ]:
figure = plt.figure(figsize=(20,10))
sns.relplot(x="E_nu/GeV_max", y="A_Eff_cm2", sizes=(40, 400), alpha=.2, color="r",height=10, data=data)
plt.show()

In [ ]:
figure = plt.figure(figsize=(20,10))
sns.relplot(x="Dec_nu_min_deg", y="A_Eff_cm2", sizes=(40, 400), alpha=.2, color="r",height=10, data=data)
plt.show()

In [ ]:
figure = plt.figure(figsize=(20,10))
sns.relplot(x="Dec_nu_max_deg", y="A_Eff_cm2", sizes=(40, 400), alpha=.2, color="r",height=10, data=data)
plt.show()

In [ ]:
figure = plt.figure(figsize=(20,8))
sns.boxplot(x="Dec_nu_max_deg",y="A_Eff_cm2",data=data)
plt.show()

In [ ]:
figure = plt.figure(figsize=(20,8))
sns.boxplot(x="Dec_nu_min_deg",y="A_Eff_cm2",data=data)
plt.show()

In [ ]:
figure = plt.figure(figsize=(20,8))
sns.boxplot(x="E_nu/GeV_max",y="A_Eff_cm2",data=data)
plt.show()

In [ ]:
figure = plt.figure(figsize=(20,8))
sns.boxplot(x="E_nu/GeV_min",y="A_Eff_cm2",data=data)
plt.show()

In [ ]:
fig = plt.figure(figsize=(20,10))
ax = Axes3D(fig)
ax.scatter(data["Dec_nu_min_deg"], data["E_nu/GeV_min"], data["A_Eff_cm2"], c="red", s=20, alpha=0.2)
plt.show()

In [ ]:
fig = plt.figure(figsize=(20,10))
ax = Axes3D(fig)
ax.scatter(data["Dec_nu_max_deg"], data["E_nu/GeV_max"], data["A_Eff_cm2"], c="red", s=20, alpha=0.2)
plt.show()

In [ ]:
fig = plt.figure(figsize=(20,10))
ax = Axes3D(fig)
ax.scatter(data["Dec_nu_max_deg"], data["E_nu/GeV_min"], data["A_Eff_cm2"], c="red", s=20, alpha=0.2)
plt.show()

In [ ]:
fig = plt.figure(figsize=(20,10))
median = np.median(data["E_nu/GeV_max"])
plt.hist(data["Dec_nu_max_deg"], edgecolor="red",color="black")
plt.axvline(median,color="green")
plt.show()

In [ ]:
fig = plt.figure(figsize=(20,10))
median = np.median(data["E_nu/GeV_max"])
plt.hist(data["Dec_nu_min_deg"], edgecolor="red",color="black")
plt.axvline(median,color="green")
plt.show()

In [ ]:
fig = plt.figure(figsize=(20,10))
sns.pairplot(data,hue="A_Eff_cm2")
plt.show()

# AGAINST VALUES

In [ ]:
DataForC = data.copy()

In [ ]:
clf = LocalOutlierFactor()
clf.fit_predict(DataForC)

In [ ]:
score = clf.negative_outlier_factor_
sortscore = np.sort(score)
print(sortscore[0:50])

In [ ]:
point = sortscore[4]
print(DataForC[score == point])

In [ ]:
againstvalues = DataForC < point
print(DataForC[againstvalues].value_counts().sum())

# MODELS

## X & Y FOR MODELS

In [ ]:
x = data.drop("A_Eff_cm2",axis=1)
y = data["A_Eff_cm2"]

xTrain,xTest,yTrain,yTest = train_test_split(x,y,test_size=0.2,random_state=42)

## REGRESSION MODELS

In [ ]:
lm = LinearRegression().fit(xTrain,yTrain)
pls = PLSRegression().fit(xTrain,yTrain)
ridge = Ridge().fit(xTrain,yTrain)
lasso = Lasso().fit(xTrain,yTrain)
elasticnet = ElasticNet().fit(xTrain,yTrain)
knnr = KNeighborsRegressor().fit(xTrain,yTrain)
cartr = DecisionTreeRegressor(random_state=42).fit(xTrain,yTrain)
baggr = BaggingRegressor(random_state=42,bootstrap_features=True,verbose=False).fit(xTrain,yTrain)
rfr = RandomForestRegressor(random_state=42,verbose=False).fit(xTrain,yTrain)
gbmr = GradientBoostingRegressor(verbose=False).fit(xTrain,yTrain)
xgbr = XGBRegressor().fit(xTrain,yTrain)
lgbmr = LGBMRegressor().fit(xTrain,yTrain)
catbr = CatBoostRegressor(verbose=False).fit(xTrain,yTrain)

In [ ]:
models = [lm,pls,ridge,lasso,elasticnet,knnr,
         cartr,baggr,rfr,gbmr,xgbr,lgbmr,catbr]

In [ ]:
for model in models:
    name = model.__class__.__name__
    R2CV = cross_val_score(model,xTest,yTest,cv=10,scoring="r2").mean()
    error = -cross_val_score(model,xTest,yTest,cv=10,scoring="neg_mean_squared_error").mean()
    print(name + ": ")
    print("-" * 10)
    print(R2CV)
    print(np.sqrt(error))
    print("-" * 30)

In [ ]:
r = pd.DataFrame(columns=["MODELS","R2CV"])
for model in models:
    name = model.__class__.__name__
    R2CV = cross_val_score(model,xTest,yTest,cv=10,scoring="r2").mean()
    result = pd.DataFrame([[name,R2CV*100]],columns=["MODELS","R2CV"])
    r = r.append(result)
    
figure = plt.figure(figsize=(20,8))   
sns.barplot(x="R2CV",y="MODELS",data=r,color="k")
plt.xlabel("R2CV")
plt.ylabel("MODELS")
plt.xlim(0,100)
plt.title("MODEL ACCURACY COMPARISON")
plt.show()

In [ ]:
r = pd.DataFrame(columns=["MODELS","error"])
for model in models:
    name = model.__class__.__name__
    error = -cross_val_score(model,xTest,yTest,cv=10,scoring="neg_mean_squared_error").mean()
    result = pd.DataFrame([[name,np.sqrt(error)]],columns=["MODELS","error"])
    r = r.append(result)
    
figure = plt.figure(figsize=(20,8))   
sns.barplot(x="error",y="MODELS",data=r,color="r")
plt.xlabel("Error")
plt.ylabel("MODELS")
plt.xlim(0,100000000)
plt.title("MODEL ERROR COMPARISON")
plt.show()

## OLS

In [ ]:
ols = sm.OLS(yTrain,yTrain).fit()

print(ols.summary())

## PCR

In [ ]:
pca = PCA()
xRTrain = pca.fit_transform(scale(xTrain))
xRTest = pca.fit_transform(scale(xTest))

In [ ]:
lmP = LinearRegression().fit(xRTrain,yTrain)
R2CV = cross_val_score(lmP,xRTest,yTest,cv=10,scoring="r2").mean()
error = -cross_val_score(lmP,xRTest,yTest,cv=10,scoring="neg_mean_squared_error").mean()

print(R2CV)
print(np.sqrt(error))

## ANN

In [ ]:
scaler = StandardScaler().fit(xTrain,yTrain)
xRTrain = scaler.transform(xTrain)
xRTest = scaler.transform(xTest)

In [ ]:
mlpr = MLPRegressor().fit(xTrain,yTrain)

R2CV = cross_val_score(mlpr,xRTest,yTest,cv=10,scoring="r2").mean()
error = -cross_val_score(mlpr,xRTest,yTest,cv=10,scoring="neg_mean_squared_error").mean()

print(R2CV)
print(np.sqrt(error))